In [1]:
from goatools.base import get_godag
import pandas as pd
import requests

go = get_godag("go-basic.obo", optional_attrs={"relationship"})

from Similarity import Similarity_of_Two_GOTerms, Similarity_of_Set_of_GOTerms

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2024-06-17) 45,494 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2024-06-17) 45,494 Terms; optional_attrs(relationship)


In [2]:
go_id1 = "GO:0005634"
go_id2 = "GO:0042579"
go_id3 = "GO:0005777"
go_id4 = "GO:0031903"

In [3]:
method_list = ["GOntoSim", "Baseline", "Baseline_LCA", "Baseline_Desc", "wang", "GOGO"]


def Similarity_of_Set_of_GOTerms(go_id1, go_id2, go, method):
    print(method)
    print(
        go_id1,
        "and",
        go_id2,
        "=",
        Similarity_of_Two_GOTerms(go_id1, go_id2, go, method),
    )


for method in method_list:
    Similarity_of_Set_of_GOTerms(go_id1, go_id2, go, method)
    Similarity_of_Set_of_GOTerms(go_id3, go_id4, go, method)

GOntoSim
GO:0005634 and GO:0042579 = 0.43452661328308995
GOntoSim
GO:0005777 and GO:0031903 = 0.7516592341352092
Baseline
GO:0005634 and GO:0042579 = 0.7246229452635147
Baseline
GO:0005777 and GO:0031903 = 0.6725006629541236
Baseline_LCA
GO:0005634 and GO:0042579 = 0.8690532265661799
Baseline_LCA
GO:0005777 and GO:0031903 = 0.9131545338441889
Baseline_Desc
GO:0005634 and GO:0042579 = 0.36231147263175734
Baseline_Desc
GO:0005777 and GO:0031903 = 0.6313322986901766
wang
GO:0005634 and GO:0042579 = 0.7439251158246399
wang
GO:0005777 and GO:0031903 = 0.6888698543860534
GOGO
GO:0005634 and GO:0042579 = 0.8088116501220667
GOGO
GO:0005777 and GO:0031903 = 0.7966598157740756


In [4]:
gene1 = [
    "GO:0004022",
    "GO:0004024",
    "GO:0004174",
    "GO:0046872",
    "GO:0008270",
    "GO:0004023",
    "GO:0016491",
]


gene2 = ["GO:0009055", "GO:0005515", "GO:0046872", "GO:0008270", "GO:0020037"]

In [5]:
method = "GOntoSim"
print(method)
print(Similarity_of_Set_of_GOTerms(gene1, gene2, method))

method = "Baseline"
print(method)
# print(Similarity_of_Set_of_GOTerms(['GO:0004022','GO:0004024', 'GO:0004174', 'GO:0046872','GO:0008270','GO:0004023', 'GO:0016491'],['GO:0009055','GO:0005515','GO:0046872','GO:0008270','GO:0020037'], method))
print(Similarity_of_Set_of_GOTerms(gene1, gene2, method))


method = "Baseline_LCA"
print(method)
# print(Similarity_of_Set_of_GOTerms(['GO:0004022','GO:0004024', 'GO:0004174', 'GO:0046872','GO:0008270','GO:0004023', 'GO:0016491'],['GO:0009055','GO:0005515','GO:0046872','GO:0008270','GO:0020037'], method))
print(Similarity_of_Set_of_GOTerms(gene1, gene2, method))


method = "Baseline_Desc"
print(method)
# print(Similarity_of_Set_of_GOTerms(['GO:0004022','GO:0004024', 'GO:0004174', 'GO:0046872','GO:0008270','GO:0004023', 'GO:0016491'],['GO:0009055','GO:0005515','GO:0046872','GO:0008270','GO:0020037'], method))
print(Similarity_of_Set_of_GOTerms(gene1, gene2, method))


method = "wang"
print(method)
# print(Similarity_of_Set_of_GOTerms(['GO:0004022','GO:0004024', 'GO:0004174', 'GO:0046872','GO:0008270','GO:0004023', 'GO:0016491'],['GO:0009055','GO:0005515','GO:0046872','GO:0008270','GO:0020037'], method))
print(Similarity_of_Set_of_GOTerms(gene1, gene2, method))


method = "GOGO"
print(method)
# print(Similarity_of_Set_of_GOTerms(['GO:0004022','GO:0004024', 'GO:0004174', 'GO:0046872','GO:0008270','GO:0004023', 'GO:0016491'],['GO:0009055','GO:0005515','GO:0046872','GO:0008270','GO:0020037'], method))
print(Similarity_of_Set_of_GOTerms(gene1, gene2, method))

GOntoSim
0.683
Baseline
0.423
Baseline_LCA
1.004
Baseline_Desc
0.432
wang
0.536
GOGO
0.53


In [6]:
def getGoTerms(URL):
    requestURL = (
        "https://www.ebi.ac.uk/QuickGO/services/annotation/downloadSearch?geneProductId="
        + URL
    )
    r = requests.get(requestURL, headers={"Accept": "text/tsv"})
    if not r.ok:
        responseBody = "!Too many parameters\n"
        return responseBody
        # r.raise_for_status()
        # sys.exit()

    responseBody = r.text
    return responseBody


def get_Annotations(protein, aspect="MF"):
    annotation = getGoTerms(protein)
    # print(annotation)
    df = pd.DataFrame(
        [x.split("\t") for x in annotation.split("\n")[1:]],
        columns=[x for x in annotation.split("\n")[0].split("\t")],
    )
    df = df.drop(
        [
            "GENE PRODUCT DB",
            "ECO ID",
            "REFERENCE",
            "WITH/FROM",
            "TAXON ID",
            "ASSIGNED BY",
            "ANNOTATION EXTENSION",
        ],
        axis=1,
    )

    if aspect == "MF":
        Filtered = df[(df["GO ASPECT"] == "F")]
    if aspect == "BP":
        Filtered = df[(df["GO ASPECT"] == "P")]
    if aspect == "CC":
        Filtered = df[(df["GO ASPECT"] == "C")]
    # Drop rows with duplicate GO terms
    Filtered = Filtered.drop_duplicates("GO TERM")
    x = Filtered["GO TERM"].tolist()
    return x

# Similarity of 2 Proteins

In [7]:
protein1 = "P0CG30"
protein1_GOTerms = get_Annotations(protein1, "MF")

protein2 = "Q9P6M1"
protein2_GOTerms = get_Annotations(protein2, "MF")

method = "GOntoSim"
print(method)
print(Similarity_of_Set_of_GOTerms(protein1_GOTerms, protein2_GOTerms, method))

GOntoSim
0.759
